In [2]:
from scrape_courses import get_courses_from_dgscene, get_readable_course_name
from scrape_events import get_all_sanctioned_events
from scrape_ratings import get_round_ratings_for_tournament
from logger import logger
import json
import itertools

/home/tbeidlershenk/.local/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# TODO
# generate list of all courses
# write to json
courses = get_courses_from_dgscene()
course_names = {
    course: get_readable_course_name(course) for course in courses
}
with open('data/course_names.json', 'w') as f:
    json.dump(course_names, f, indent=4)

In [2]:
with open('data/course_names.json') as f:
    course_names: dict = json.load(f)

with open('data/course_events.json') as f:
    course_events: dict = json.load(f)

for i, course in enumerate(course_names):
    if course in course_events:
        logger.info(f'Skipping {course} (already scraped)...')
        continue

    logger.info(f'Fetching event {i}/{len(course_names)}')
    course_events[course] = get_all_sanctioned_events(course)
    logger.info(course_events[course])

    # periodically save ratings to file
    with open('data/course_events.json', 'w') as f:
        json.dump(course_events, f, indent=4)

logger.info("Done")

[20/Oct/2024 18:02:29] INFO - Skipping 22_Birdwalk (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping 4_Wing_Campground (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Alexander_Hamilton_Park (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Anderson_Park_AB (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Aperture_Park (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Aspen_Meadows (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Badlands_DGC (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Baileys_Crossing_Disc_Golf_Course (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Baker_Park_AB (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Barrett_Park (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Bear_Lake_Open (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping bear-paw-par-3-12330 (already scraped)...
[20/Oct/2024 18:02:29] INFO - Skipping Bonavista_Downs_Community (alrea

In [4]:
try:
    logger.info('Fetching ratings...')
    with open('data/course_events.json') as f:
        course_events: dict = json.load(f)
    try:
        with open('data/course_ratings.json', 'r') as f:
            course_ratings: dict = json.load(f)
    except FileNotFoundError:
        course_ratings = {}

    for i, course in enumerate(course_events):
        if course in course_ratings:
            logger.info(f'Skipping {course} (already scraped)...')
            continue
        course_ratings[course] = {}
        events = course_events[course]

        for j, event in enumerate(events):
            event_id = event['event_id']
            course_ratings[course][event_id] = get_round_ratings_for_tournament(event_id)
            logger.info(
                f'Event {j+1}/{len(events)} - Course {i+1}/{len(course_events)}')

        # periodically save ratings to file
        with open('data/course_ratings.json', 'w') as f:
            json.dump(course_ratings, f, indent=4)

except BaseException as e:
    logger.info(f'Error fetching ratings: {e.with_traceback()}')
except KeyboardInterrupt as e:
    logger.info(f'Error fetching ratings: {e.with_traceback()}')

logger.info("Done")

[21/Oct/2024 14:48:31] INFO - Fetching ratings...
[21/Oct/2024 14:48:32] INFO - Skipping 22_Birdwalk (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping 4_Wing_Campground (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping Alexander_Hamilton_Park (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping Anderson_Park_AB (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping Aperture_Park (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping Aspen_Meadows (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping Badlands_DGC (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping Baileys_Crossing_Disc_Golf_Course (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping Baker_Park_AB (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping Barrett_Park (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping Bear_Lake_Open (already scraped)...
[21/Oct/2024 14:48:32] INFO - Skipping bear-paw-par-3-12330 (already scraped)...
[21/Oct/2024 14:48:32

In [ ]:
logger.info('Compiling data...')
with open('data/course_names.json') as f:
    course_names: dict = json.load(f)
with open('data/course_events.json') as f:
    course_events: dict = json.load(f)
with open('data/course_ratings.json') as f:
    course_ratings: dict = json.load(f)

course_data = {}
for course in course_names:
    rounds = []
    for event_id, round_data in course_ratings.get(course, {}).items():
        round_data.put('event_id', event_id)
        round_data.put(course_events)
        rounds.extend(round_data)
    course_data[course] = {
        'name': course_names[course],
        'rounds': list(itertools.chain.from_iterable([round_data for event_id, round_data in course_ratings.get(course, {}).items()]))
    }

course_data = {
    course: {
        'name': course_names[course],
        'rounds': list(itertools.chain.from_iterable([round_data for event_id, round_data in course_ratings.get(course, {}).items()]))
    } for course in course_names
}

with open('data/course_data.json', 'w') as f:
    json.dump(course_data, f, indent=4)
    
logger.info("Done")
